Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

In [1]:
import numpy as np
import random
from itertools import product
from collections import Counter
import logging

logging.basicConfig(
    format="[%(asctime)s] %(levelname)s: %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

In [2]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

In [4]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

def eval_terminal(board):
    if four_in_a_row(board, 1):
        return 1
    if four_in_a_row(board, -1):
        return -1
    return 0

In [5]:
def current_player(board):
    if (board != 0).sum() % 2 == 0:
        return 1
    return -1

In [ ]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 1000
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)

In [68]:
def minmax(board, max_depth=1):
    plyer = current_player(board)
    if max_depth == 0:
        return None, eval_board(board, plyer)
    max_depth = max_depth - 1 
    branches = list()
    for ply in valid_moves(board):
        play(board, ply, plyer)
        val = eval_board(np.copy(board), plyer)
        branches.append((ply, -val))
        take_back(board, ply)
    print(branches)
    return max(branches, key=lambda k: k[1])

In [11]:
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
print(board)
p = 1
c = 0
for _ in range(NUM_COLUMNS*(COLUMN_HEIGHT-2)):
    c = random.choice(valid_moves(board))
    play(board, c, p)
    if four_in_a_row(board, p):
        print(p, "has 4 in row")
        break
    p = -p
    
print(board)
print("move to", p)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[ 1  1 -1  0  0  0]
 [-1  1 -1 -1  1  0]
 [ 1 -1 -1  1 -1  1]
 [ 1  1  1 -1  0  0]
 [-1 -1 -1  0  0  0]
 [ 1  1 -1 -1  0  0]
 [-1  1  1  0  0  0]]
move to 1


In [72]:
board

array([[ 1,  1, -1,  0,  0,  0],
       [-1,  1, -1, -1,  1,  0],
       [ 1, -1, -1,  1, -1,  1],
       [ 1,  1,  1, -1,  0,  0],
       [-1, -1, -1,  0,  0,  0],
       [ 1,  1, -1, -1,  0,  0],
       [-1,  1,  1,  0,  0,  0]], dtype=int8)

In [73]:
print(board)
best, val = minmax(board)
p = current_player(board)
print(p, ':', best, val)

[[ 1  1 -1  0  0  0]
 [-1  1 -1 -1  1  0]
 [ 1 -1 -1  1 -1  1]
 [ 1  1  1 -1  0  0]
 [-1 -1 -1  0  0  0]
 [ 1  1 -1 -1  0  0]
 [-1  1  1  0  0  0]]
[(0, 0.6276), (1, 0.4466), (3, 0.7748), (4, 0.4314), (5, 0.636), (6, 0.2562)]
1 : 3 0.7748


In [71]:
test_cases = [
    np.array([[ 1,  1, -1,  0,  0,  0],
       [-1,  1, -1, -1,  1,  0],
       [ 1, -1, -1,  1, -1,  1],
       [ 1,  1,  1, -1,  0,  0],
       [-1, -1, -1,  0,  0,  0],
       [ 1,  1, -1, -1,  0,  0],
       [-1,  1,  1,  0,  0,  0]], dtype=np.int8)]

board = test_cases[0]